## Testing the **Single Index** vector DB

### Data Preprocessing

1. Load the **PDFs or Markdown** files in Order
2. Extract the **data/content** from the PDF
3. Then perform chunking - *Because the context window for the LLMs are small*
4. Then pass it to the **Embedding Models**

In [44]:
# 1. Loading of the PDFs

from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

In [45]:
import os

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
DATA_FILE_PATH = os.path.join(parent_dir, "data")

In [46]:
# Function to load PDF files from a specified directory
def load_pdf_file(file_path=DATA_FILE_PATH):
    if not os.path.isdir(file_path):
        print(f"Error: The provided path '{file_path}' is not a directory.")
        return []

    loader = DirectoryLoader(
        file_path,
        glob="*.pdf",
        loader_cls=PyPDFLoader,
        recursive=True,
    )

    documents = loader.load()
    return documents


# Function to load Markdown files from a specified directory
# This function doesn't extract all the required data.
def load_markdown_file(file_path=DATA_FILE_PATH):
    if not os.path.isdir(file_path):
        print(f"Error: The provided path '{file_path}' is not a directory.")
        return []

    loader = DirectoryLoader(
        file_path,
        glob="**/*.md",           
        loader_cls=TextLoader,
        loader_kwargs={"encoding": "utf-8"}
    )

    docs = loader.load()
    return docs

In [47]:
import frontmatter
from pathlib import Path

# This will be the Function to load MARKDOWN FILES
def load_md_with_metadata(path):
    docs = []

    for file in Path(path).rglob("*.md"):
        post = frontmatter.load(file)

        docs.append(
            Document(
                page_content=post.content,
                metadata={
                    "title": post.get("title"),
                    "description": post.get("description"),
                    "source": str(file)
                }
            )
        )

    return docs

In [48]:
import re
import markdown
from bs4 import BeautifulSoup

# Function to clean markdown text
def clean_markdown(md_text: str) -> str:
    # 1. Remove YAML front matter
    md_text = re.sub(r"^---.*?---", "", md_text, flags=re.DOTALL)

    # 2. Convert markdown → HTML
    html = markdown.markdown(md_text)

    # 3. Parse HTML
    soup = BeautifulSoup(html, "html.parser")

    # 4. Remove unwanted tags
    for tag in soup(["script", "style", "iframe", "img", "table"]):
        tag.decompose()

    # 5. Get text
    text = soup.get_text(separator=" ")

    # 6. Remove markdown links but keep text
    text = re.sub(r"\[(.*?)\]\(.*?\)", r"\1", text)

    # 7. Normalize whitespace
    text = re.sub(r"\n{2,}", "\n\n", text)
    text = re.sub(r"[ \t]+", " ", text)

    return text.strip()

In [49]:
# Filters documents to only include page_content and source metadata.
def filter_to_minimal_docs(docs):
    minimal_docs = []

    for doc in docs:
        full_path = doc.metadata.get("source", "")
        file_name = os.path.basename(full_path)

        minimal_doc = Document(
            page_content = clean_markdown(doc.page_content),
            metadata={
                "title": doc.metadata.get("title"),
                "description": doc.metadata.get("description"),
                "source": file_name
            }
        )

        minimal_docs.append(minimal_doc)

    return minimal_docs

In [50]:
# Split the documents into smaller chunks
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=100
    )
    split_docs = text_splitter.split_documents(minimal_docs)
    return split_docs

In [106]:
# Loading the Markdown files
docs = load_md_with_metadata(DATA_FILE_PATH)

# Filtering to only include page_content and source metadata.
minimal_docs = filter_to_minimal_docs(docs)

# Split the minimized documents into text chunks
text_chunks = text_split(minimal_docs)

## Performing the Vector Embedding on Text Data:

The Embedding Model is **Sentence-Transformers**

1. The chunks are processed and converted to vectors
2. Then the embedded vectors is then sent to the Endee Vector Database using the Endee API
3. Once the Data is stored in the the Vector DB, it is now ready to serve as a Knowledge base for the LLM

In [52]:
from sentence_transformers import SentenceTransformer
embeddingModel = SentenceTransformer('all-MiniLM-L6-v2')

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 176.28it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


### Storing the vector embeddings for each text chunk

In [108]:
start_idx = 1
vectorEmbeddings = []

for id, chunk in enumerate(text_chunks):
    source = chunk.metadata.get('source', "")
    title = chunk.metadata.get("title", "")
    description = chunk.metadata.get("description", "")
    text = chunk.page_content
    embedding = embeddingModel.encode(text).tolist()

    data = {
        "id": id + start_idx,
        "vector": embedding,
        "meta": {
            "title": title,
            "description": description,
            "source": source,
            "text": text
        }
    }
    vectorEmbeddings.append(data)

### Setting up for connecting with Endee API powered by Flask

In [110]:
import requests
from requests.exceptions import ConnectionError, Timeout, HTTPError

# Index name for Endee Vector Database
INDEX_NAME = "enterprise_knowledge_base2"

# URL for Endee API service
ENDEE_URL = "http://127.0.0.1:8000" 

### Creating a Index in Endee Vector Database

Using only the Dense Vectors (***Vector Embeddings***)

In [113]:
# Payload for creating an index in Endee Vector DB
payload = {
    "index_name": INDEX_NAME,
    "dimension": embeddingModel.get_sentence_embedding_dimension(),
    "precision": "INT16D"
}

In [114]:
try:    
    response = requests.post(
        f"{ENDEE_URL}/index/create",
        json=payload
    )
    response.raise_for_status()
    print(f"Message for index creation: {response.json()}")

except ConnectionError:
    print("Backend service is not reachable (is it running?)")
    
except Timeout:
    print("Request timed out")
    
except HTTPError as e:
    try:
        err = e.response.json()
        print("Error message:", err.get("error"))
    except ValueError:
        print("Raw error:", e.response.text)
    
except Exception as e:
    print(f"Unexpected error: {e}")

Message for index creation: {'index_name': 'enterprise_knowledge_base2', 'status': 'index created'}


### Checking for the Existence of the Index in the Endee Vector Database

In [115]:
try:
    response = requests.post(
        f"{ENDEE_URL}/index/get",
        json={
            "index_name": INDEX_NAME
        })
    
    response.raise_for_status()
    print(response.json())
    
except ConnectionError:
    print("Backend service is not reachable (is it running?)")
    
except Timeout:
    print("Request timed out")
    
except HTTPError as e:
    try:
        err = e.response.json()
        print("Error message:", err.get("error"))
    except ValueError:
        print("Raw error:", e.response.text)
    
except Exception as e:
    print(f"Unexpected error: {e}")

{'index_name': 'enterprise_knowledge_base2', 'status': 'index loaded'}


### Inserting the Embedded Vectors into Endee Vector DB 

In [ ]:
# Single query/chunk
payload = {
    "index_name": INDEX_NAME,
    "embedded_vectors": [vectors]
}

try:
    response = requests.post(
        f"{ENDEE_URL}/index/upsert",
        json=payload
    )

    response.raise_for_status()
    print(response.json())
    
except ConnectionError:
    print("Backend service is not reachable (is it running?)")
    
except Timeout:
    print("Request timed out")
    
except HTTPError as e:
    try:
        err = e.response.json()
        print("Error message:", err.get("error"))
    except ValueError:
        print("Raw error:", e.response.text)
    
except Exception as e:
    print(f"Unexpected error: {e}")

Error message: list indices must be integers or slices, not str


In [116]:
# Performing list slicing because, insertion limit is 1000 vectors, and I am keeping 950 vectors per upsert
slices = []

end = len(vectorEmbeddings) // 950
prev = 0
for i in range(1, end + 1):
    slices.append((prev, 950 * i + 1))
    prev = 950 * i + 1
slices.append((prev, len(vectorEmbeddings)))

In [118]:
# This is the function to UPSERT the VECTORS into DB
def upsertVectors(vectors):
    payload = {
            "index_name": INDEX_NAME,
            "embedded_vectors": vectors
    }

    try:
        response = requests.post(
            f"{ENDEE_URL}/index/upsert",
            json=payload
        )
        response.raise_for_status()

        print(response.json())
        return True

    except ConnectionError:
        print("Backend service is not reachable (is it running?)")

    except Timeout:
        print("Request timed out")

    except HTTPError as e:
        try:
            err = e.response.json()
            print("Error message:", err.get("error"))
        except ValueError:
            print("Raw error:", e.response.text)

    except Exception as e:
        print(f"Unexpected error: {e}")

    return False

In [119]:
for start, end in slices:
    vectors = vectorEmbeddings[start:end]
    
    if not upsertVectors(vectors):
        break

{'count': 951, 'status': 'vectors upserted'}
{'count': 950, 'status': 'vectors upserted'}
{'count': 950, 'status': 'vectors upserted'}
{'count': 950, 'status': 'vectors upserted'}
{'count': 950, 'status': 'vectors upserted'}
{'count': 741, 'status': 'vectors upserted'}


### Retrieving the Top K most relevant data

In [ ]:
# Sample RETRIEVAL of data relevant to queries.
query = "Who is Sayan Mondal"
embedding_for_query = embeddingModel.encode(query).tolist()

payload = {
    "index_name": INDEX_NAME,
    "vector": embedding_for_query,
    "top_k": 20
}

# Sends a query to the Endee API
response = requests.post(
    f"{ENDEE_URL}/index/query",
    json=payload
)

data = response.json()
print(data["results"][:2])

[{'description': 'Learn more about working with Manav Khurana, CPMO at GitLab.', 'distance': 0.7510538846254349, 'id': '3176', 'similarity': 0.24894611537456512, 'text': 'Manav Khurana is the current Chief Product and Marketing Officer \n About Me \n Manav Kurana is a business-minded technical person. Two things drive my work practice: \n \n \n Solving customer problems – more of them and doing so in the best way. \n \n \n Optimizing for business results in what we do.', 'title': 'Manav Khurana - Chief Product and Marketing Officer - README'}, {'description': 'Learn more about working with Manav Khurana, CPMO at GitLab.', 'distance': 0.759822741150856, 'id': '3178', 'similarity': 0.24017725884914398, 'text': 'Manav Kurana was born and raised in New Delhi, India. Manav Kurana came to the U.S. to study electrical and computer engineering in New York and then moved to San Francisco for what’s now been a 25-year career in tech. My wife, whom Manav Kurana met in college, is a psychologist a

### Build complete RAG Pipeline:

1. Using GROQ Cloud for it's Free APIs and Fast response
2. Using Langchain as a warapper

### How  does this work:

1. `retriever`: returns `List[Document]` for a query (Find relevant information)
2. `prompt`: a prompt template with `{context}` and `{input}` (Frame the question for the AI)
3. `chatModel`: an LLM runnable (Generate the answer)
4. `RunnablePassthrough`: Carry the user’s question forward unchanged
5. `StrOutputParser`: Clean the output

In [69]:
# Loading the environment variables from the .env file
from dotenv import load_dotenv
load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_groq import ChatGroq

In [ ]:
llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    temperature=0.3
)

In [124]:
# This is wrapper for retrieving the data
def endee_retriever(query: str):
    embedding_for_query = embeddingModel.encode(query).tolist()

    payload = {
        "index_name": INDEX_NAME,
        "vector": embedding_for_query,
        "top_k": 20
    }

    response = requests.post(
        f"{ENDEE_URL}/index/query",
        json=payload
    )

    response.raise_for_status()
    data = response.json()

    docs = []
    for d in data.get("results", []):
        text = d.get("text", "")

        docs.append(
            Document(
                page_content=text,
                metadata={
                    "similarity": d.get("similarity"),
                    "source": d.get("source"),
                    "title": d.get("title"),
                    "description": d.get("description")
                }
            )
        )

    return docs

retriever = RunnableLambda(endee_retriever)

In [148]:
system_prompt = """
You are a professional and a helpful virtual assistant for a Company named GitLab.
Your name is 'GitLab Copilot' and your job is to answer to all the queries made by the employees of this company to make their work easier.
Answer in three to five sentences for general questions.
and if the answer needs to be more elaborated, provide more details like step by step instructions only when needed or asked specifically by the user.
Use the context given below for your reference and keep the answer concise and to the point.
Respond in detail only when the user specifies it.
And always perform a Double check before giving the final response to the user.

Context:
{context}

If a user ask whether they can upload a Document, respond with "Yes, you can upload a PDF Document only."
"""

In [ ]:
# Building the Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

# Rag pipeline:
rag_chain = (
    {
        "context": retriever | (lambda docs: "\n\n".join(d.page_content for d in docs)),
        "input": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [77]:
# Sample test:
print(rag_chain.invoke("If I want to prepare for an Interview, what all should I need to learn, can you guide me in this. Tell me step by step in detail"))

To prepare for an interview, here's a step-by-step guide:

1. **Review the Job Description**: Study the job description and requirements to understand the skills and qualifications needed for the position.
2. **Research the Company**: Learn about the company's mission, values, products, and services to demonstrate your interest and knowledge.
3. **Update Your Resume and Online Profiles**: Ensure your resume and online profiles (e.g., LinkedIn) are up-to-date and highlight your relevant skills and experiences.
4. **Prepare Common Interview Questions**: Familiarize yourself with common interview questions, such as "Why do you want to work for this company?" or "What are your strengths and weaknesses?"
5. **Practice Your Responses**: Prepare thoughtful responses to these questions, using the STAR method ( Situation, Task, Action, Result) to structure your answers.
6. **Develop Questions to Ask the Interviewer**: Prepare a list of questions to ask the interviewer, such as "What are the big

In [ ]:
print(rag_chain.invoke("what is the use of CES in Team Interviews and also tell what is CES?, give me in points"))

CES stands for Candidate Experience Specialist. Here are the key points about CES and its use in Team Interviews:

* CES is responsible for scheduling technical interviews with candidates.
* CES utilizes tools like ModernLoop and GitLab Service Desk to track and manage interview requests.
* Key responsibilities of CES in Team Interviews include:
  * Scheduling interviews with team members
  * Sending interview confirmations to candidates
  * Coordinating with interviewers and candidates to find suitable interview times
  * Ensuring a smooth and efficient interview process
* CES follows a specific process for scheduling interviews, including:
  * Receiving availability requests from candidates
  * Sending interview invites to interviewers
  * Confirming interview details with candidates
* CES also handles non-standard requests and escalations, and maintains communication with candidates and interviewers throughout the interview process.


In [ ]:
# Answers only when relevant data is retrieved
print(rag_chain.invoke("Who is Sayan Mondal"))

There is no information about Sayan Mondal in the provided context. If you have any other questions or need assistance with something else, feel free to ask.


In [153]:
# Testing the RAG pipeline with user input in a loop
while True:
    user_input = input("User: ")
    print(f"User: {user_input}")
    if user_input.lower() in ["exit", "quit"]:
        print("\nExiting the chat. Goodbye!!!")
        break

    response = rag_chain.invoke(user_input)
    print(f"Assistant: {response}\n")

User: Hi
Assistant: Hello, I'm GitLab Copilot. How can I assist you today?

User: WHo are you
Assistant: I'm GitLab Copilot, a professional and helpful virtual assistant for GitLab. I'm here to answer your queries and make your work easier.

User: What can you do for me?
Assistant: I'm GitLab Copilot, your virtual assistant. I can help answer your queries, provide guidance, and make your work easier. If you have any questions or need assistance with anything related to GitLab, feel free to ask, and I'll do my best to help. Whether it's about our values, workflows, or technical aspects, I'm here to support you. What's on your mind?

User: How can I get a job in this company
Assistant: To apply for a job at GitLab, you can visit the internal job openings page. Additionally, you can join the #newvacancy channel in Slack to stay updated on new openings. If you have any questions about the hiring process, you can create an issue in the hiring process repo or reach out to a Recruiter directl